In [1]:
import duckdb

In [2]:
DATABASE = "../database.db"

In [120]:
with duckdb.connect(DATABASE) as con:
    con.sql("""
    WITH
    output AS (
        SELECT
            r.report_id,
            r.player_id,
            uga.week,
            uga.team,
            uga.opponent AS opp,
            vga.week AS prev,
            vga.opponent AS prev_opp,
            vgo.rushing_attempts AS prev_att,
            ugo.rushing_attempts AS att,
            CASE
                WHEN att IS NULL OR prev_att IS NULL THEN 'unknown'
                WHEN att - prev_att > 6 THEN 'much_higher'
                WHEN att - prev_att > 3 THEN 'somewhat_higher'
                WHEN prev_att - att > 6 THEN 'much_lower'
                WHEN prev_att - att > 3 THEN 'somewhat_lower'
                ELSE 'similar'
            END AS actual,
            rp.expected_workload_change AS expect,
        FROM comparable_report c
        LEFT JOIN report r
            ON c.report_id = r.report_id
        LEFT JOIN report_game_assignment a
            ON r.report_id = a.report_id
        LEFT JOIN game ug
            ON a.upcoming_game_id = ug.game_id
        LEFT JOIN player_game_assignment uga
            ON a.upcoming_game_id = uga.game_id
            AND r.player_id = uga.player_id
        LEFT JOIN player_game_assignment vga
            ON a.previous_game_id = vga.game_id
            AND r.player_id = vga.player_id
        LEFT JOIN player_game_outcome ugo
            ON uga.game_id = ugo.game_id
            AND uga.player_id = ugo.player_id
        LEFT JOIN player_game_outcome vgo
            ON vga.game_id = vgo.game_id
            AND vga.player_id = vgo.player_id
        LEFT JOIN report_prediction rp
            ON r.report_id = rp.report_id
        ORDER BY
            ugo.season ASC,
            r.player_id ASC,
            ugo.week ASC
    )
    SELECT *
    FROM output
    LIMIT 20
    ;
    """).show()

┌───────────────────┬───────────┬───────┬─────────┬─────────┬───────┬──────────┬──────────┬───────┬─────────────────┬─────────────────┐
│     report_id     │ player_id │ week  │  team   │   opp   │ prev  │ prev_opp │ prev_att │  att  │     actual      │     expect      │
│      varchar      │   int64   │ int64 │ varchar │ varchar │ int64 │ varchar  │  int64   │ int64 │     varchar     │     varchar     │
├───────────────────┼───────────┼───────┼─────────┼─────────┼───────┼──────────┼──────────┼───────┼─────────────────┼─────────────────┤
│ rotoballer_201562 │      4147 │     2 │ CIN     │ JAX     │     1 │ CLE      │        0 │     0 │ similar         │ much_lower      │
│ rotoballer_202245 │      4147 │     3 │ CIN     │ MIN     │     2 │ JAX      │        0 │     4 │ somewhat_higher │ much_higher     │
│ rotoballer_202391 │      4147 │     4 │ CIN     │ DEN     │     3 │ MIN      │        4 │     1 │ similar         │ somewhat_lower  │
│ rotoballer_202303 │      4147 │     4 │ CIN   